# Argon code oud (alleen tijdsevolutie)

### Define constants and parameters first

In [1]:
import numpy as np
import math
from numba import jit
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# constants - used to calculate endresults in SI. Computation is in natural units, i.e. m=1 eps=1 sigma=1.
kb = 1.38 * 10**(-23) #J/K
eps_per_kb = 120 #K
sigma = 0.34*10**(-9) #m
m = 6.633853*10**(-26) #kg - mass of argon atom
tau = math.sqrt(m*sigma*sigma/eps_per_kb/kb) #s - typical unit of time.

#### ----------------------------------- Parameters
# adjustable params (Natural units!)
rho = 0.75                #width of the box in sigma
Temp = 1                 #For fixed energy computation, must be scalar
n = 3                     #Number of unit cells per dimension
dt = 0.004                #In natural units
timesteps = 15          #number of timesteps

# computed params
N = 4*n*n*n #Number of particles
L = (N/rho)**(1/3) 

### Initial position computation

In [2]:
def init_pos():
    v = np.linspace(0,1,n,False)
    w = np.linspace(1/(2*n),(2*n-1)/(2*n),n,True)
    
    X1,Y1,Z1 = np.meshgrid(v,v,v) #corners
    X2,Y2,Z2 = np.meshgrid(w,w,v) #face centers per axis
    X3,Y3,Z3 = np.meshgrid(v,w,w)
    X4,Y4,Z4 = np.meshgrid(w,v,w)
    x = np.hstack((X1.reshape(n**3),X2.reshape(n**3),X3.reshape(n**3),X4.reshape(n**3)))
    y = np.hstack((Y1.reshape(n**3),Y2.reshape(n**3),Y3.reshape(n**3),Y4.reshape(n**3)))
    z = np.hstack((Z1.reshape(n**3),Z2.reshape(n**3),Z3.reshape(n**3),Z4.reshape(n**3)))
    
    return L*x, L*y, L*z

### Initial velocity computation

In [3]:
def init_vel():
    v = np.random.normal(0,math.sqrt(Temp),(N,3))
    vx = v[:,0] - v[:,0].mean()
    vy = v[:,1] - v[:,1].mean()
    vz = v[:,2] - v[:,2].mean()
    return vx,vy,vz

### Force computation based on LJ potential

In [4]:
# Force between particle i and j
@jit
def forceLJ(xi,yi,zi,xj,yj,zj):
    dx = xi-(xj + np.around((xi-xj)/L)*L) 
    dy = yi-(yj + np.around((yi-yj)/L)*L) 
    dz = zi-(zj + np.around((zi-zj)/L)*L) 
    r2 = dx*dx + dy*dy + dz*dz
    pref = 48/(r2**7) - 24/(r2**4)
    Fx = pref*dx
    Fy = pref*dy
    Fz = pref*dz
    V = 4/(r2**6) - 4/(r2**3)
    return Fx, Fy, Fz, V

# Total force on each particle in a vector, and total potential energy of all particles.
@jit
def forceTotal(x,y,z):
    Fx = np.zeros(N)
    Fy = np.zeros(N)
    Fz = np.zeros(N)
    V = 0.0
    for i in range(N):
        for j in range(i):
            dFx,dFy,dFz,dV = forceLJ(x[i],y[i],z[i],x[j],y[j],z[j])
            Fx[i] += dFx
            Fy[i] += dFy
            Fz[i] += dFz
            # action = - reaction
            Fx[j] -= dFx
            Fy[j] -= dFy
            Fz[j] -= dFz
            
            V += dV
    return Fx,Fy,Fz,V

### Evolve in time function

In [5]:
def evolveTimeConserveE(x,y,z,vx,vy,vz,numtime):
    #Initialize kinetic and potential energy vectors
    K = np.zeros(numtime)
    V = np.zeros(numtime)
    Fx,Fy,Fz,discard = forceTotal(x,y,z)
    for i in range(numtime):
        vx += 0.5*dt*Fx
        vy += 0.5*dt*Fy
        vz += 0.5*dt*Fz        
        
        x += dt*vx
        y += dt*vy
        z += dt*vz
        x = x%L
        y = y%L
        z = z%L

        Fx,Fy,Fz,V[i] = forceTotal(x,y,z)
        
        vx += dt/2*Fx
        vy += dt/2*Fy
        vz += dt/2*Fz  
        
        K[i] = 0.5*np.sum(vx*vx+vy*vy+vz*vz)
       
    return x,y,z,vx,vy,vz,K,V

### Main program that calls the evolve function

In [12]:
%%timeit
x,y,z = init_pos()
vx,vy,vz = init_vel()
x,y,z,vx,vy,vz,K,V = evolveTimeConserveE(x,y,z,vx,vy,vz,timesteps)

100 loops, best of 3: 4.76 ms per loop


In [10]:
fig = plt.figure(1)
plt.plot(range(timesteps),K,'b',range(timesteps),V,'g',range(timesteps),K+V,'r')
plt.legend(('Kinetic', 'Potential', 'Total'))
plt.show()